<a href="https://colab.research.google.com/github/RhudsonDouglas/ods16-deteccao-armas-dashboard./blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision scikit-learn matplotlib pandas -q
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
# Métricas
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm.notebook import tqdm

In [ ]:
# Clonar o repositório
!git clone https://github.com/poori-nuna/HOD-Benchmark-Dataset.git
# Classe do Dataset
class HODDataset(Dataset):
def __init__(self, base_path, classes, transform=None, include_subsets=("normal
self.base_path = base_path
self.classes = classes
self.transform = transform
self.samples = []
valid_ext = (".jpg", ".jpeg", ".png", ".bmp")
for cls in classes:
for subset in include_subsets:
subset_path = os.path.join(base_path, cls, subset, "jpg")
if os.path.exists(subset_path):
imgs = [f for f in os.listdir(subset_path) if f.lower().endswit
for f in imgs:
img_path = os.path.join(subset_path, f)
self.samples.append((img_path, cls))
self.class_to_idx = {c: i for i, c in enumerate(sorted(classes))}
self.idx_to_class = {i: c for c, i in self.class_to_idx.items()} # Facilita
def __len__(self):
return len(self.samples)
def __getitem__(self, idx):
img_path, cls = self.samples[idx]
image = Image.open(img_path).convert("RGB")
label = self.class_to_idx[cls]
if self.transform:
image = self.transform(image)
return image, label

Para o treino, usamos Data Augmentation (cortes aleatórios, inversão) para o modelo generalizar
melhor. Para validação, usamos apenas o resize e normalização.

In [ ]:
# Caminho base
base_path = "HOD-Benchmark-Dataset/dataset/class"
classes = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path
print(f"Classes encontradas: {classes}")
# Normalização padrão do ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
# Transforms de treino (com Augmentation) e validação (simples)
data_transforms = {
'train': transforms.Compose([
# ALTERAÇÃO: Substituído RandomResizedCrop por Resize + RandomCrop para evi
# Redimensiona a imagem para o tamanho desejado (128x128), garantindo o ges
transforms.Resize((128, 128)),
# Aplica um corte aleatório de 128x128, com padding opcional de 4 pixels.
# Isso simula pequenas variações na posição do gesto dentro do quadro.
transforms.RandomCrop(128, padding=4),
# ADIÇÃO: Ajusta brilho, contraste, saturação e hue aleatoriamente.
# ALTERAÇÃO: Adicionada probabilidade de 50% para esta transformação.
transforms.RandomApply([
transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hu
], p=0.5), # Probabilidade de 50%
# ADIÇÃO: Aplica rotação, translação, escala e cisalhamento aleatoriamente.
# ALTERAÇÃO: Adicionada probabilidade de 40% para esta transformação.
transforms.RandomApply([
transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1
], p=0.4), # Probabilidade de 40%
transforms.RandomHorizontalFlip(), # Mantido. Por padrão, p=0.5 (50%).
# ADIÇÃO: Aplica distorção de perspectiva aleatória.
# ALTERAÇÃO: Adicionada probabilidade de 20% para esta transformação.
transforms.RandomApply([
transforms.RandomPerspective(distortion_scale=0.5)
], p=0.2), # Probabilidade de 20%
transforms.ToTensor(),
normalize
]),
'val': transforms.Compose([
transforms.Resize((128, 128)),
transforms.ToTensor(),
normalize
]),
}

Visualização de amostras das imagens transformadas

In [ ]:
# --- Bloco para Visualizar Imagens Transformadas ---
# Certifique-se de que a classe HODDataset está definida e que os paths estão corre
# Execute esta célula DEPOIS da célula onde você define 'full_dataset', 'data_trans
def visualize_transforms(dataset, classes, num_images_per_class=2):
"""
Visualiza um pequeno conjunto de imagens de cada classe após a aplicação das tr
Args:
dataset (Dataset): O dataset de treino (Subset) com a transform de treino a
classes (list): Lista de nomes das classes.
num_images_per_class (int): Número de imagens a visualizar por classe.
"""
# Mapeamento de índice para nome da classe do dataset original
idx_to_class = {i: cls_name for cls_name, i in dataset.dataset.class_to_idx.ite
# Criar um dicionário para armazenar índices de imagens por classe
# Estes índices são os índices DENTRO do subconjunto (train_dataset)
class_indices_in_subset = {i: [] for i in range(len(classes))}
# Iterar sobre os índices do SUBSET para obter os rótulos
# dataset.indices contém os índices do dataset original que estão neste subset
for subset_idx in range(len(dataset)):
# Obtém o índice original do dataset
original_idx = dataset.indices[subset_idx]
# Obtém a amostra do dataset original para pegar o rótulo
# Não aplicamos a transform aqui, só queremos o rótulo
_, cls = dataset.dataset.samples[original_idx]
label = dataset.dataset.class_to_idx[cls]
if label in class_indices_in_subset:
class_indices_in_subset[label].append(subset_idx) # Armazena o índice D
fig, axes = plt.subplots(len(classes), num_images_per_class, figsize=(num_image
fig.suptitle('Imagens após Transforms de Treino', fontsize=16, y=1.02)
for class_idx, cls_name in enumerate(classes):
# Pega índices aleatórios para a classe atual DENTRO do subset
# Garantindo que não pegamos mais imagens do que existem
if not class_indices_in_subset[class_idx]: # Skip if no images for this cla
print(f"Warning: No images found for class '{cls_name}' in the training
https://colab.research.google.com/drive/1wJ3sZjeSiRsnm-F8ECHnJm6Sg2KWbIy9?usp=sharing#scrollTo=oJ9eCfE7wRWI&printM…
3/905/11/2025, 11:21
Projetos 5 - Etapa 2 (Treinamento) - Colab
continue
selected_subset_indices = np.random.choice(
class_indices_in_subset[class_idx],
min(num_images_per_class, len(class_indices_in_subset[class_idx])),
replace=False
)
for i, subset_idx in enumerate(selected_subset_indices):
# Obtém a imagem e o label (já transformada) usando o índice DO SUBSET
image_tensor, label_idx = dataset[subset_idx]
# Desnormaliza a imagem para exibição (reverte a normalização do ImageN
# Valores padrão do ImageNet: mean=[0.485, 0.456, 0.406], std=[0.229, 0
# O tensor está em [C, H, W], precisamos para [H, W, C] para matplotlib
image_np = image_tensor.numpy().transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
image_np = std * image_np + mean # Desnormaliza
image_np = np.clip(image_np, 0, 1) # Garante que os valores estejam ent
ax = axes[class_idx, i] if len(classes) > 1 else axes[i]
ax.imshow(image_np)
ax.set_title(f"{cls_name}", fontsize=10)
ax.axis('off')
plt.tight_layout(rect=[0, 0, 1, 0.98]) # Ajusta layout para o suptitle
plt.show()
# --- Chame a função para visualizar ---
# Certifique-se de que 'train_dataset' já tem a transform de treino aplicada!
print("Visualizando algumas imagens transformadas do conjunto de treino:")
visualize_transforms(train_dataset, classes, num_images_per_class=3) # Altere num_i

ResNet-50 pré-treinada no ImageNet.tods os pesos foram congelados e a última camada trocada
(fc) por uma nova, com o número de saídas igual ao número de classes do dataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
# Carregar ResNet-50 pré-treinada
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
# Congelar os pesos da rede (só vamos treinar a camada final por enquanto)
for param in model.parameters():
param.requires_grad = False
# Substituir a camada final (fc)
num_ftrs = model.fc.in_features
num_classes = len(classes)
model.fc = nn.Linear(num_ftrs, num_classes)
# Mover o modelo para a GPU
model = model.to(device)
print("Modelo ResNet-50 carregado e camada final substituída.")


para organizar o loop de treino e validaçã

In [ ]:
# Loss (para classificação)
criterion = nn.CrossEntropyLoss()
# Otimizador (Fase 1: treina SÓ os parâmetros da camada fc)
optimizer_head = optim.Adam(model.fc.parameters(), lr=0.001)
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=5
history_per_iteration = {'train_loss_iter': [], 'train_acc_iter': [], 'val_loss
history_per_epoch = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_ac
best_acc = 0.0
global_iteration = 0
for epoch in range(num_epochs):
# --- Treino ---
model.train()
running_loss = 0.0
running_corrects = 0
total_samples = 0
train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Train)"
for i, (inputs, labels) in enumerate(train_bar):
inputs, labels = inputs.to(device), labels.to(device)
optimizer.zero_grad()
with torch.set_grad_enabled(True):
outputs = model(inputs)
_, preds = torch.max(outputs, 1)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()
current_loss = loss.item() * inputs.size(0)
current_corrects = torch.sum(preds == labels.data)
running_loss += current_loss
running_corrects += current_corrects
total_samples += inputs.size(0)
# Registrar histórico por iteração
history_per_iteration['train_loss_iter'].append(loss.item())
history_per_iteration['train_acc_iter'].append(current_corrects.item()
global_iteration += 1
train_bar.set_postfix(loss=loss.item(), acc=current_corrects.item() / i
epoch_train_loss = running_loss / total_samples
epoch_train_acc = running_corrects.double() / total_samples
history_per_epoch['train_loss'].append(epoch_train_loss)
history_per_epoch['train_acc'].append(epoch_train_acc.item())
# --- Validação ---
model.eval()
running_loss = 0.0
running_corrects = 0
total_samples = 0
val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)
with torch.no_grad():
for inputs, labels in val_bar:
inputs, labels = inputs.to(device), labels.to(device)
outputs = model(inputs)
_, preds = torch.max(outputs, 1)
loss = criterion(outputs, labels)
current_loss = loss.item() * inputs.size(0)
current_corrects = torch.sum(preds == labels.data)
running_loss += current_loss
running_corrects += current_corrects
total_samples += inputs.size(0)
# Registrar histórico por iteração (opcional para validação, mas in
history_per_iteration['val_loss_iter'].append(loss.item())
history_per_iteration['val_acc_iter'].append(current_corrects.item(
val_bar.set_postfix(loss=loss.item(), acc=current_corrects.item() /
epoch_val_loss = running_loss / total_samples
epoch_val_acc = running_corrects.double() / total_samples
history_per_epoch['val_loss'].append(epoch_val_loss)
history_per_epoch['val_acc'].append(epoch_val_acc.item())
print(f"
Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_train_loss:.4f}
# Opcional: Salvar o melhor modelo
if epoch_val_acc > best_acc:
best_acc = epoch_val_acc
# torch.save(model.state_dict(), 'best_model.pth') # Descomente para sa
print("Treinamento concluído!")
return model, history_per_epoch, history_per_iteration


treino foi rodado por algumas épocas. Como o optimizer_head só conhece os parâmetros de
model.fc, apenas essa camada será atualizada

In [ ]:
print("--- Iniciando Treinamento (Fase 1: Só o classificador) ---")
num_epochs_head = 5 # 5 épocas é um bom começo
model, history_head_epoch, history_head_iteration = train_model(
model,
criterion,
optimizer_head,
train_loader,
test_loader, # Usando test_loader para validação durante o treino
num_epochs_head,
device
)
print("\n--- Treinamento (Fase 1) Concluído ---")

a rede inteira será descongelada e um novo otimizador será criado (optimizer_all) que otimiza todos
os parâmetros. Usamos um learning rate (lr) bem baixo para não estragar os pesos pré-treinados,
apenas ajustá-los (fine-tuning).

In [ ]:
print("--- Iniciando Treinamento (Fase 2: Fine-Tuning completo) ---")
# 1. Descongelar a rede
for param in model.parameters():
param.requires_grad = True
# 2. Novo otimizador para todos os parâmetros, com lr baixo
optimizer_all = optim.Adam(model.parameters(), lr=1e-5) # lr = 0.00001
num_epochs_all = 5 # Mais 5 épocas de ajuste fino
model, history_all_epoch, history_all_iteration = train_model(
model,
criterion,
optimizer_all,
train_loader,
test_loader, # Usando test_loader para validação durante o treino
num_epochs_all,
device
)

Para fechar a Etapa 2, rodamos o modelo no conjunto de teste e plotamos a matriz de confusão

In [ ]:
print("--- Iniciando Avaliação Final ---")
y_pred = []
y_true = []
model.eval() # Modo de avaliação
with torch.no_grad():
for inputs, labels in tqdm(test_loader, desc="Gerando Predições"):
inputs, labels = inputs.to(device), labels.to(device)
outputs = model(inputs)
_, preds = torch.max(outputs, 1)
y_pred.extend(preds.cpu().numpy())
y_true.extend(labels.cpu().numpy())
print("\n--- Matriz de Confusão ---")